Guide: https://wiki.postgresql.org/wiki/Psycopg2_Tutorial

In [1]:
import psycopg2
from pandas import DataFrame as df

In [2]:
try:
    conn = psycopg2.connect("dbname='phil' user='phil' host='localhost'")
except:
    print("I am unable to connect to the database")

In [3]:
cur = conn.cursor()

In [4]:
cur.execute("""SELECT * from cards""")

In [5]:
query_data = df.from_records(cur.fetchall(),
                               columns = [desc[0] for desc in cur.description])

In [6]:
query_data

,id,board_id,data
0,1,1,"{'name': 'Paint house', 'tags': ['Improvements..."
1,2,1,"{'name': 'Wash dishes', 'tags': ['Clean', 'Kit..."
2,3,1,"{'name': 'Cook lunch', 'tags': ['Cook', 'Kitch..."
3,4,1,"{'name': 'Vacuum', 'tags': ['Clean', 'Bedroom'..."
4,5,1,"{'name': 'Hang paintings', 'tags': ['Improveme..."
5,6,1,"{'name': 'Paint house', 'tags': ['Improvements..."


In [7]:
query_data.at[0, 'data']

{'name': 'Paint house', 'tags': ['Improvements', 'Office'], 'finished': True}

In [8]:
type(query_data.at[0, 'data'])

dict

In [9]:
type(query_data.at[0, 'data']['name'])

str

In [10]:
type(query_data.at[0, 'data']['tags'])

list

In [11]:
type(query_data.at[0, 'data']['finished'])

bool

In [12]:
cur.execute("""SELECT * from cards""")

In [13]:
query_data = cur.fetchall()

In [14]:
query_data

[(1,
  1,
  {'name': 'Paint house',
   'tags': ['Improvements', 'Office'],
   'finished': True}),
 (2,
  1,
  {'name': 'Wash dishes', 'tags': ['Clean', 'Kitchen'], 'finished': False}),
 (3,
  1,
  {'name': 'Cook lunch',
   'tags': ['Cook', 'Kitchen', 'Tacos'],
   'finished': False,
   'ingredients': ['Tortillas', 'Guacamole']}),
 (4,
  1,
  {'name': 'Vacuum',
   'tags': ['Clean', 'Bedroom', 'Office'],
   'finished': False}),
 (5,
  1,
  {'name': 'Hang paintings',
   'tags': ['Improvements', 'Office'],
   'finished': False}),
 (6,
  1,
  {'name': 'Paint house',
   'tags': ['Improvements', 'Office'],
   'finished': True})]

In [15]:
type(query_data)

list

In [16]:
query_data[0][2]

{'name': 'Paint house', 'tags': ['Improvements', 'Office'], 'finished': True}

In [17]:
type(query_data[0][2])

dict

In [18]:
from jinjasql import JinjaSql
import json
j = JinjaSql()

In [19]:
template = """
    INSERT INTO cards 
    VALUES (6, 1, {{ json_data }});
"""

In [20]:
data = { "json_data": json.dumps(query_data[0][2])}

In [21]:
type(data["json_data"])

str

In [22]:
query, bind_params = j.prepare_query(template, data)

In [23]:
query

'\n    INSERT INTO cards \n    VALUES (6, 1, %s);'

In [24]:
bind_params

odict_values(['{"name": "Paint house", "tags": ["Improvements", "Office"], "finished": true}'])

#### We'll need to cast the `odict_values` into a `list` for it to work in the query.

In [25]:
cur.execute(query, list(bind_params))

In [26]:
#The commit line will commit the exectued query to the db
#conn.commit()

In [27]:
query_data[5][2]

{'name': 'Paint house', 'tags': ['Improvements', 'Office'], 'finished': True}

In [28]:
type(query_data[5][2])

dict